# BER regression analysis

This notebook fits a regression model to available BER data in order to build a general cost estimator for any BER rating and any house size. The data used in this notebook was sourced from [SEAI](https://www.seai.ie/publications/Your-Guide-to-Building-Energy-Rating.pdf).

Start by importing the packages we'll need:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression

Load the data to analyse:

In [ ]:
data = [
    ['A1', 75, 0.4, 140],
    ['A2', 75, 0.8, 280],
    ['A3', 75, 1.0, 350],
    ['B1', 75, 1.3, 440],
    ['B2', 75, 1.6, 570],
    ['B3', 75, 2.0, 700],
    ['C1', 75, 2.4, 800],
    ['C2', 75, 2.8, 1000],
    ['C3', 75, 3.2, 1100],
    ['D1', 75, 3.7, 1300],
    ['D2', 75, 4.4, 1500],
    ['E1', 75, 5.0, 1800],
    ['E2', 75, 5.7, 2000],
    ['F', 75, 6.8, 2400],
    ['G', 75, 8.5, 3000],
    ['A1', 100, 0.5, 190],
    ['A2', 100, 1.1, 380],
    ['A3', 100, 1.4, 470],
    ['B1', 100, 1.7, 590],
    ['B2', 100, 2.2, 800],
    ['B3', 100, 2.7, 900],
    ['C1', 100, 3.1, 1100],
    ['C2', 100, 3.7, 1300],
    ['C3', 100, 4.2, 1500],
    ['D1', 100, 5.0, 1700],
    ['D2', 100, 5.8, 2000],
    ['E1', 100, 6.7, 2300],
    ['E2', 100, 7.6, 2600],
    ['F', 100, 9.1, 3200],
    ['G', 100, 11.3, 4000],
    ['A1', 150, 0.8, 280],
    ['A2', 150, 1.6, 560],
    ['A3', 150, 2.0, 700],
    ['B1', 150, 2.5, 900],
    ['B2', 150, 3.3, 1100],
    ['B3', 150, 4.0, 1400],
    ['C1', 150, 4.7, 1600],
    ['C2', 150, 5.5, 1900],
    ['C3', 150, 6.3, 2200],
    ['D1', 150, 7.5, 2600],
    ['D2', 150, 8.8, 3100],
    ['E1', 150, 10.1, 3500],
    ['E2', 150, 11.4, 4000],
    ['F', 150, 13.6, 4700],
    ['G', 150, 17.0, 5900],
    ['A1', 200, 1.1, 400],
    ['A2', 200, 2.2, 800],
    ['A3', 200, 2.7, 900],
    ['B1', 200, 3.4, 1200],
    ['B2', 200, 4.3, 1500],
    ['B3', 200, 5.3, 1900],
    ['C1', 200, 6.3, 2200],
    ['C2', 200, 7.4, 2600],
    ['C3', 200, 8.4, 2900],
    ['D1', 200, 10.0, 3500],
    ['D2', 200, 11.7, 4100],
    ['E1', 200, 13.4, 4700],
    ['E2', 200, 15.1, 5300],
    ['F', 200, 18.2, 6300],
    ['G', 200, 22.7, 7900],
    ['A1', 300, 1.6, 600],
    ['A2', 300, 3.2, 1100],
    ['A3', 300, 4.1, 1400],
    ['B1', 300, 5.0, 1800],
    ['B2', 300, 6.5, 2300],
    ['B3', 300, 8.0, 2800],
    ['C1', 300, 9.4, 3300],
    ['C2', 300, 11.0, 3900],
    ['C3', 300, 12.7, 4400],
    ['D1', 300, 14.9, 5200],
    ['D2', 300, 17.5, 6100],
    ['E1', 300, 20.1, 7000],
    ['E2', 300, 22.7, 7900],
    ['F', 300, 27.2, 9500],
    ['G', 300, 34, 11900]
]

df = pd.DataFrame(data, columns=['ber', 'area', 'co2', 'cost'])

Fit a simple regression model:

In [ ]:
X = pd.get_dummies(df[['ber']]).mul(df['area'], axis='index')
y = df['cost']

model = LinearRegression(fit_intercept=False)
model.fit(X, y)

error = model.predict(X) - y
print(f'MAE: {np.mean(np.abs(error))}')
print(f'MSE: {np.mean(error ** 2)}')

plt.hist(error, bins=20)
plt.gca().set(
    xlabel='Error',
    ylabel='Frequency'
)
sns.despine()

Take a look at the parameters:

In [ ]:
parameters = pd.Series(dict(zip(X.columns, model.coef_)))
parameters

Create a Google Sheets query:

In [ ]:
query = '=SWITCH($B3, '
for k, v in parameters.items():
    query += f'"{k[4:]}", {v} * C$2, '
query = query[:-2] + ')'

query